In [1]:
from pennylane import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import torch

from feature_map import *
from ansatz import *

In [2]:
def generate_chebyshev_grid(n, a=0, b=0.9, requires_grad=True):
    k = torch.arange(n)
    chebyshev_nodes = torch.cos(torch.pi * (2*k + 1) / (2*n))  #standard Chebyshev nodes in [-1,1]
    scaled_nodes = ((chebyshev_nodes + 1) / 2) * (b - a) + a  #rescale to [a, b]
    
    return scaled_nodes.requires_grad_(requires_grad=requires_grad)

N_POINTS = 10
x = generate_chebyshev_grid(N_POINTS, requires_grad=False).detach().numpy()
print(x)

[0.8944597  0.85095286 0.7681981  0.6542957  0.5203955  0.37960446
 0.24570423 0.13180195 0.04904703 0.00554024]


In [3]:
#x = np.linspace(0, 0.9, 20)
#print(x)
#np.random.seed(42)
#theta_init = np.array(2*np.pi*np.random.random(num_qubits*d*3), requires_grad=True)
qadence_init = [0.05815449, 0.9552357, 0.84251973, 0.62340308, 0.03134017, 0.25185522, 0.06291017, 0.92875258, 0.87050104, 0.437279, 0.04036647, 0.23340819, 0.12358607, 0.08354337, 0.37753296, 0.07473488, 0.93190879, 0.2256651, 0.05258016, 0.13264067, 0.61243659, 0.68343022, 0.15212583, 0.3175678, 0.5261719, 0.1570535, 0.08810978, 0.31223868, 0.26504491, 0.61440828, 0.47678475, 0.37536703, 0.70118189, 0.30905878, 0.13043349, 0.59777135]

theta_init = np.array(qadence_init, requires_grad=True)
print(theta_init)

[0.05815449 0.9552357  0.84251973 0.62340308 0.03134017 0.25185522
 0.06291017 0.92875258 0.87050104 0.437279   0.04036647 0.23340819
 0.12358607 0.08354337 0.37753296 0.07473488 0.93190879 0.2256651
 0.05258016 0.13264067 0.61243659 0.68343022 0.15212583 0.3175678
 0.5261719  0.1570535  0.08810978 0.31223868 0.26504491 0.61440828
 0.47678475 0.37536703 0.70118189 0.30905878 0.13043349 0.59777135]


### f(x)

In [4]:
num_qubits = 6
d = 2

def total_magnetization(num_qubits):
    return sum(qml.PauliZ(i) for i in range(num_qubits))
def create_circuit(x, num_qubits, d, theta, label):

    U(x, num_qubits, label)
    qml.Barrier()
    HEA(theta, num_qubits, d)
    cost_op = total_magnetization(num_qubits) 
   
    return qml.expval(cost_op)

In [5]:
label = 0
point = x[9]

dev = qml.device('default.qubit', wires=list(range(num_qubits)))
circuit = qml.QNode(create_circuit, dev)
result = circuit(point, num_qubits, d, theta_init, label)

print(result)
print(qml.draw(circuit)(point, num_qubits, d, theta_init, label))

-0.5827048206133318
0: ──RY(1.57)──||──RZ(0.06)──RX(0.96)──RZ(0.84)─╭●──RZ(0.62)──RX(0.03)──RZ(0.25)───────────╭●────┤
1: ──RY(3.13)──||──RZ(0.06)──RX(0.93)──RZ(0.87)─╰X─╭●─────────RZ(0.44)──RX(0.04)──RZ(0.23)─╰X─╭●─┤
2: ──RY(4.70)──||──RZ(0.12)──RX(0.08)──RZ(0.38)─╭●─╰X─────────RZ(0.07)──RX(0.93)──RZ(0.23)─╭●─╰X─┤
3: ──RY(6.26)──||──RZ(0.05)──RX(0.13)──RZ(0.61)─╰X─╭●─────────RZ(0.68)──RX(0.15)──RZ(0.32)─╰X─╭●─┤
4: ──RY(7.83)──||──RZ(0.53)──RX(0.16)──RZ(0.09)─╭●─╰X─────────RZ(0.31)──RX(0.27)──RZ(0.61)─╭●─╰X─┤
5: ──RY(9.39)──||──RZ(0.48)──RX(0.38)──RZ(0.70)─╰X──RZ(0.31)──RX(0.13)──RZ(0.60)───────────╰X────┤

  ╭<Z+Z+Z+Z+Z+Z>
  ├<Z+Z+Z+Z+Z+Z>
  ├<Z+Z+Z+Z+Z+Z>
  ├<Z+Z+Z+Z+Z+Z>
  ├<Z+Z+Z+Z+Z+Z>
  ╰<Z+Z+Z+Z+Z+Z>
